In [77]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 
import numpy as np
from tensorflow.contrib.layers import batch_norm 
# batch_norm(...): Adds a Batch Normalization layer from http://arxiv.org/abs/1502.03167

In [78]:
# 构件图模型
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_hidden3 = 50
n_outputs = 10

In [79]:
# 提取数据
mnist = input_data.read_data_sets("MNIST_data_bak/")

Extracting MNIST_data_bak/train-images-idx3-ubyte.gz
Extracting MNIST_data_bak/train-labels-idx1-ubyte.gz
Extracting MNIST_data_bak/t10k-images-idx3-ubyte.gz
Extracting MNIST_data_bak/t10k-labels-idx1-ubyte.gz


In [80]:
# 构建输入层节点
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None,), name='y')
# 与上次同样，X为(None,784)的矩阵，y为一维的矩阵

In [81]:
# 构建神经网络，我们有两个隐藏层，全连接，激活函数为Relu
def neron_layer(X, n_neurons, name, activation=None):
    #模块化每层神经元的编写，name_scope可不写
    with tf.name_scope(name):
        # 取输入矩阵的维度作为层的输入连接个数
        n_inputs = int(X.get_shape()[1]) # X.get_shape()[1]获取矩阵的行数
        stddev = 2 / np.sqrt(n_inputs)   # 初始化方差
        # 每两个层之间的网络对应一个 w 的二维数组
        # truncated normal distribution 比 regular normal distribution的值小
        # 不会出现任何大的权重值，确保慢慢的稳健的训练
        # 使用这种标准方差会让收敛快
        # w参数需要随机，不能为0，否则输出为0，最后调整都是一个幅度没意义
        init = tf.truncated_normal((n_inputs,n_neurons), stddev=stddev) 
        # Outputs random values from a truncated normal distribution.初始化w矩阵，初始化分布为stddev
        W = tf.Variable(init, name="weights") # 将 w push 到 节点中去
        B = tf.Variable(tf.zeros([n_neurons]), name="biases") # push 截距项
        z = tf.matmul(X,W) + B
        # X:(?, n_inputs) W:(n_inputs,n_neurons) 
        # XW:(?, n_neurons) B:([n_neurons])
        # XW+B (?,n_neurons[index]+n_neurons[index])
        if activation == "relu":
            return tf.nn.relu(z) # Computes rectified linear: max(features, 0)
        else:
            return z

In [82]:
with tf.name_scope("dnn"):
    hidden1 = neron_layer(X, n_hidden1, "hidden1", activation = "relu")      # 784 * 300
    hidden2 = neron_layer(hidden1, n_hidden2, "hidden2", activation="relu")   # 300 * 100
    hidden3 = neron_layer(hidden2, n_hidden3, "hidden2", activation="relu")   # 100 * 50
    # 以上为进入到输出层之前的结果
    logits = neron_layer(hidden3, n_outputs, "outputs")  # 50 * 10

In [83]:
with tf.name_scope("loss"):
    # 定义交叉熵损失函数，并求平均样本
    Xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    # Computes sparse softmax cross entropy between logits and labels.
    # 函数等价于先使用softmax损失函数，再接着计算交叉熵，并且更有效率(数学上的优化)
    # sparse_： 对y稀释，转成one_hot码 
    loss = tf.reduce_mean(Xentropy, name="loss")

In [84]:
# 学习率
learning_rate = 0.01

In [85]:
with tf.name_scope("train"):
    # 利用 tensorflow 提供的自动梯度下降 迭代
    training_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss) 

In [86]:
with tf.name_scope("eval"):
    # 获取logits里面最大的那一位和y比较类别好是否相同，返回True或者False一组值
    correct = tf.nn.in_top_k(logits, y, 1)
    # Says whether the targets are in the top K predictions.
    # 在logits(?行，10列)的数组中，最大的索引是for是y的值，是返回True，否返回false
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    # 先转换类型，后计算reduce_mean

In [87]:
init = tf.global_variables_initializer()

In [88]:
# 计算图阶段
n_epochs = 400
batch_size = 50

In [89]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run([training_op], feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                            y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.92 Test accuracy: 0.9206
1 Train accuracy: 0.9 Test accuracy: 0.9341
2 Train accuracy: 0.94 Test accuracy: 0.9469
3 Train accuracy: 0.96 Test accuracy: 0.9543
4 Train accuracy: 1.0 Test accuracy: 0.9551
5 Train accuracy: 0.98 Test accuracy: 0.9597
6 Train accuracy: 0.96 Test accuracy: 0.9641
7 Train accuracy: 1.0 Test accuracy: 0.9636
8 Train accuracy: 1.0 Test accuracy: 0.9647
9 Train accuracy: 1.0 Test accuracy: 0.9677
10 Train accuracy: 1.0 Test accuracy: 0.9675
11 Train accuracy: 1.0 Test accuracy: 0.9702
12 Train accuracy: 1.0 Test accuracy: 0.9675
13 Train accuracy: 1.0 Test accuracy: 0.9675
14 Train accuracy: 0.98 Test accuracy: 0.9704
15 Train accuracy: 1.0 Test accuracy: 0.9713
16 Train accuracy: 0.98 Test accuracy: 0.972
17 Train accuracy: 1.0 Test accuracy: 0.9714
18 Train accuracy: 1.0 Test accuracy: 0.972
19 Train accuracy: 1.0 Test accuracy: 0.9738
20 Train accuracy: 1.0 Test accuracy: 0.9729
21 Train accuracy: 1.0 Test accuracy: 0.9726
22 Train accura

181 Train accuracy: 1.0 Test accuracy: 0.9765
182 Train accuracy: 1.0 Test accuracy: 0.9767
183 Train accuracy: 1.0 Test accuracy: 0.9769
184 Train accuracy: 1.0 Test accuracy: 0.9769
185 Train accuracy: 1.0 Test accuracy: 0.9768
186 Train accuracy: 1.0 Test accuracy: 0.977
187 Train accuracy: 1.0 Test accuracy: 0.9768
188 Train accuracy: 1.0 Test accuracy: 0.9769
189 Train accuracy: 1.0 Test accuracy: 0.9768
190 Train accuracy: 1.0 Test accuracy: 0.9766
191 Train accuracy: 1.0 Test accuracy: 0.9771
192 Train accuracy: 1.0 Test accuracy: 0.9769
193 Train accuracy: 1.0 Test accuracy: 0.9768
194 Train accuracy: 1.0 Test accuracy: 0.9767
195 Train accuracy: 1.0 Test accuracy: 0.977
196 Train accuracy: 1.0 Test accuracy: 0.9769
197 Train accuracy: 1.0 Test accuracy: 0.9769
198 Train accuracy: 1.0 Test accuracy: 0.9766
199 Train accuracy: 1.0 Test accuracy: 0.977
200 Train accuracy: 1.0 Test accuracy: 0.9769
201 Train accuracy: 1.0 Test accuracy: 0.9767
202 Train accuracy: 1.0 Test accuracy

360 Train accuracy: 1.0 Test accuracy: 0.9769
361 Train accuracy: 1.0 Test accuracy: 0.977
362 Train accuracy: 1.0 Test accuracy: 0.977
363 Train accuracy: 1.0 Test accuracy: 0.977
364 Train accuracy: 1.0 Test accuracy: 0.977
365 Train accuracy: 1.0 Test accuracy: 0.9771
366 Train accuracy: 1.0 Test accuracy: 0.977
367 Train accuracy: 1.0 Test accuracy: 0.977
368 Train accuracy: 1.0 Test accuracy: 0.9771
369 Train accuracy: 1.0 Test accuracy: 0.977
370 Train accuracy: 1.0 Test accuracy: 0.977
371 Train accuracy: 1.0 Test accuracy: 0.977
372 Train accuracy: 1.0 Test accuracy: 0.977
373 Train accuracy: 1.0 Test accuracy: 0.9769
374 Train accuracy: 1.0 Test accuracy: 0.9769
375 Train accuracy: 1.0 Test accuracy: 0.977
376 Train accuracy: 1.0 Test accuracy: 0.9769
377 Train accuracy: 1.0 Test accuracy: 0.977
378 Train accuracy: 1.0 Test accuracy: 0.9768
379 Train accuracy: 1.0 Test accuracy: 0.9769
380 Train accuracy: 1.0 Test accuracy: 0.977
381 Train accuracy: 1.0 Test accuracy: 0.977
38